In [ ]:
import pandas as pd
import os
import json

In [ ]:
data_folder = '/content/drive/MyDrive/YoutubeData/'
dataset_url= "https://www.kaggle.com/datasets/datasnaek/youtube-new?select=CAvideos.csv"


In [ ]:
# List all CSV and JSON files
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv') and 'videos' in f]
json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]

regions = ["US","GB","DE","CA","FR","RU","MX","KR","JP","IN"]
cleaned_list = []

In [ ]:
import os

files = os.listdir(data_folder)
print("Files in folder:")
for f in files:
    print("-", f)


Files in folder:
- CAvideos.csv
- DEvideos.csv
- FRvideos.csv
- GBvideos.csv
- INvideos.csv
- JPvideos.csv
- KRvideos.csv
- MXvideos.csv
- RUvideos.csv
- USvideos.csv
- RU_category_id.json
- KR_category_id.json
- JP_category_id.json
- US_category_id.json
- MX_category_id.json
- FR_category_id.json
- CA_category_id.json
- GB_category_id.json
- IN_category_id.json
- DE_category_id.json
- all_regions_cleaned.csv
- all_regions_cleaned_final.csv


In [ ]:
for csv_file in csv_files:
    # Detect region
    region = None
    for r in regions:
        if r in csv_file:
            region = r
            break
    if region is None:
        print(f"Skipping {csv_file}: region not found")
        continue

    # Corresponding JSON
    json_file = f"{region}_category_id.json"
    if json_file not in json_files:
        print(f"Skipping {csv_file}: JSON not found")
        continue

    # Load CSV
    df = pd.read_csv(os.path.join(data_folder, csv_file), encoding='latin-1', low_memory=False)

    # Load JSON categories
    with open(os.path.join(data_folder, json_file)) as f:
        data = json.load(f)
    cat_df = pd.DataFrame([{
        'category_id': int(i['id']),
        'category_name': i['snippet']['title']
    } for i in data['items']])

    # Merge categories
    df = df.merge(cat_df, how='left', on='category_id')

    # Parse dates
    df['trending_date'] = pd.to_datetime(df['trending_date'], format='%y.%d.%m', errors='coerce')
    df['publish_time'] = pd.to_datetime(df['publish_time'], errors='coerce')

    # Fill missing descriptions
    df['description'] = df['description'].fillna('')

    # Standardize numeric columns
    for c in ['views','likes','dislikes','comment_count']:
        df[c] = pd.to_numeric(df[c], errors='coerce').clip(lower=0)

    # Clean text columns
    df['title'] = df['title'].astype(str).str.strip()
    df['channel_title'] = df['channel_title'].astype(str).str.strip()

    # Remove duplicates based on video_id and trending_date
    df = df.drop_duplicates(subset=['video_id','trending_date'])

    # Add region
    df['region'] = region

    # Append to cleaned list
    cleaned_list.append(df)

# Combine all regions into a single DataFrame
if cleaned_list:
    final_df = pd.concat(cleaned_list, ignore_index=True)
    # Save final cleaned CSV
    final_df.to_csv(os.path.join(data_folder, 'all_regions_cleaned.csv'), index=False)
    print("Cleaning complete!")
    print("Regions included:", final_df['region'].unique())
    print("Total rows:", len(final_df))
else:
    print("No files were processed. Check CSVs and JSONs in your folder.")

# Optional: quick verification
print("\nDataFrame info:")
if 'final_df' in locals():
    print(final_df.info())
    print("\nMissing values per column:")
    print(final_df.isna().sum())
    print("\nDuplicates (video_id + trending_date):")
    print(final_df.duplicated(subset=['video_id','trending_date']).sum())

Cleaning complete!
Regions included: ['CA' 'DE' 'FR' 'GB' 'IN' 'JP' 'KR' 'MX' 'RU' 'US']
Total rows: 361424

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361424 entries, 0 to 361423
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   video_id                361424 non-null  object             
 1   trending_date           361424 non-null  datetime64[ns]     
 2   title                   361424 non-null  object             
 3   channel_title           361424 non-null  object             
 4   category_id             361424 non-null  int64              
 5   publish_time            361424 non-null  datetime64[ns, UTC]
 6   tags                    361424 non-null  object             
 7   views                   361424 non-null  int64              
 8   likes                   361424 non-null  int64              
 9   dislikes                361424 no

In [ ]:
final_df = final_df.drop_duplicates(subset=['video_id','trending_date'])


In [ ]:
final_df[['views','likes','dislikes','comment_count']].describe()


,views,likes,dislikes,comment_count
count,3.237300e+05,3.237300e+05,3.237300e+05,3.237300e+05
mean,1.128520e+06,2.934834e+04,1.668912e+03,3.213909e+03
std,6.993369e+06,1.360953e+05,1.838544e+04,1.927748e+04
min,1.170000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.931650e+04,5.660000e+02,3.400000e+01,9.200000e+01
50%,1.421890e+05,2.830500e+03,1.450000e+02,4.310000e+02
75%,4.967065e+05,1.355800e+04,5.910000e+02,1.620000e+03
max,4.245389e+08,5.613827e+06,1.944971e+06,1.626501e+06


In [ ]:
final_df.to_csv(os.path.join(data_folder, 'all_regions_cleaned_final.csv'), index=False)


In [ ]:
# Quick check
print("Rows after dropping duplicates:", len(final_df))
print("Missing category_name values:", final_df['category_name'].isna().sum())
print("Regions in dataset:", final_df['region'].unique())

# Show first 5 rows
final_df.head()


Rows after dropping duplicates: 323730
Missing category_name values: 2542
Regions in dataset: ['CA' 'DE' 'FR' 'GB' 'IN' 'JP' 'KR' 'MX' 'RU' 'US']


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,...,comments_disabled,ratings_disabled,video_error_or_removed,description,category_name,region,views_outlier,likes_outlier,dislikes_outlier,comment_count_outlier
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. BeyoncÃ©,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,...,False,False,False,Eminem's new track Walk on Water ft. BeyoncÃ© ...,Music,CA,1,1,1,1
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,...,False,False,False,STill got a lot of packages. Probably will las...,Comedy,CA,0,1,1,1
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,...,False,False,False,WATCH MY PREVIOUS VIDEO â¶ \n\nSUBSCRIBE âº ...,Comedy,CA,1,1,1,1
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,...,False,False,False,I know it's been a while since we did this sho...,Entertainment,CA,1,1,1,1
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,...,False,False,False,ð§: https://ad.gt/yt-perfect\nð°: https://...,Music,CA,1,1,1,1


In [ ]:
import pandas as pd

# Columns to check for outliers
num_cols = ['views', 'likes', 'dislikes', 'comment_count']

# Flag outliers using IQR method
for col in num_cols:
    Q1 = final_df[col].quantile(0.25)
    Q3 = final_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Add a new column marking outliers
    final_df[f'{col}_outlier'] = final_df[col].apply(
        lambda x: 1 if (x < lower_bound or x > upper_bound) else 0
    )

# Print how many outliers were flagged per column
for col in num_cols:
    print(f"{col} outliers:", final_df[f'{col}_outlier'].sum())


views outliers: 41882
likes outliers: 45360
dislikes outliers: 45095
comment_count outliers: 42892
